In [72]:
import urllib.request
import glob
import os
import http.server

# 1. Service of command in raspberry pi
To recevie command from distance, we build a server in the raspberry pi by http package

In [ ]:
# this cell is programmed to run in raspberry pi
PORT=8888
server_address=("",PORT)
server=http.server.HTTPServer
handler=http.server.CGIHTTPRequestHandler
handler.cgi_directories=["/"]
print("Robot server is activated on port:",PORT)
httpd.server(server_address,handler)
httpd.serve_forever()

# 2. Functions for operation of robot

## 2.1 example function of move the robot

In [ ]:
def move_robot(Robot, PWM, MOVString, TIME):
    Robot.setPWMA(PWM)
    Robot.setPWMB(PWM)
    if (MOVString=='fro' ):
        Robot.forward()
        time.sleep(TIME)
    elif (MOVString=='bac'):
        Robot.backward()
        time.sleep(TIME)
    elif (MOVString=='lef'):
        Robot.left()
        time.sleep(TIME)
    elif (MOVString=='rig'):
        Robot.right()
        time.sleep(TIME)
    else:
        Robot.stop()
    return 0

## 2.2 The program for receving command from distance

In [ ]:
# this cell is programmed to run in raspberry pi
import cgi
import time
from util_robot import *
from picamera import PiCamera
#get fields from request
form=cgi.FieldStorage()
COMMAND=form.getvalue("COMMAND")
my_list = COMMAND.split(",")
PWM = float(my_list[0])
MOVString = my_list[1]
TIME = float(my_list[2])
CAM = my_list[3]

Robot=AlphaBot2()
if CAM == 'True':
    camera = PiCamera()
    move_robot_with_camera(Robot,camera, PWM, MOVString, TIME)
else:
    move_robot(Robot, PWM, MOVString, TIME)

# 3. Sending command to robot
then we used urllib to send command to the robot
### 3.1 example movement
move the robot forword, with PMW equal to 50, running for one second, without activation of the pi camera.

In [8]:
page=urllib.urlopen("http://192.168.1.127:8888/move.py?50,fro,1,0")

### 3.2 example camera
example to take 5 images(around the camera of range 180 degrees) and get these images by wget

In [73]:
def start_get_image_from_robot():
    infile = 'http://192.168.1.121:8888/image/'
    outfile = 'keys/new_image/'
    os.system("wget --accept jpg --input-file "+ infile + " --output-file " + outfile)

In [74]:
page=urllib.urlopen("http://192.168.1.127:8888/take_image.py?5")
start_get_image_from_robot()

## 4. Refine the depth prediction
As we get the robto now, which is about 10cm on the ground. We can make sure that the image should contain a part of the ground.
So:
1. we used a classic method (without machine learning) to make a segmentation of the image.
2. take the lower parts as the groud.
3. project our result depth prediction of this segments to the ground

![title](img/imgs.PNG)

![title](img/proj1.PNG)
![title](img/proj2.PNG)
1. This method help us get a great result of the 3D representation.
2. However it gives limit to us, that we should get the information of the ground. Which may not be satisfied in real world.
3. We tested it with the robot, sometime the robot was close to some object (about 20cm), and we completely lost the detection of the ground
#### As a result, this method may not by useful